In [1]:
from codeinterpreter.sandbox.local_jupyter.manager import LocalJupyterManager

In [2]:
manager = LocalJupyterManager()
manager.init()
sandbox =  manager.start()

In [3]:
from pydantic import BaseModel
from langchain.tools import StructuredTool

def code_run(code:str):
    output =  sandbox.run(code=code)
    return output.content

class CodeInput(BaseModel):
    code: str

tools = [
    StructuredTool(
        name="python",
        description="Input a string of code to a ipython interpreter. "
        "Write the entire code in a single string. This string can "
        "be really long, so you can use the `;` character to split lines. "
        "Variables are preserved between runs. ",
        func=code_run,
        args_schema=CodeInput,
    )
]



In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
from codeinterpreter.agent.openai_functions_agent.base import CustomOpenAIFunctionsAgent


llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

agent = CustomOpenAIFunctionsAgent.from_llm_and_tools(llm=llm,tools=tools)

agent_executor = AgentExecutor(agent=agent, tools=tools, memory=ConversationBufferMemory(
                memory_key="chat_history", return_messages=True
            ), verbose=True)

In [9]:
agent_executor({"input": "print two random number"})



> Entering new AgentExecutor chain...

Invoking: `python` with `import random

# Generate two random numbers
random_number1 = random.randint(1, 100)
random_number2 = random.randint(1, 100)

# Print the random numbers
print(random_number1)
print(random_number2)`


27
11The two random numbers are 27 and 11.

> Finished chain.


{'input': 'print two random number',
 'chat_history': [HumanMessage(content='create two random number, add them with 100', additional_kwargs={}, example=False),
  AIMessage(content='The sum of two random numbers, added with 100, is 230.', additional_kwargs={}, example=False),
  HumanMessage(content='create two random number, add them with 100', additional_kwargs={}, example=False),
  AIMessage(content='The sum of two random numbers, added with 100, is 160.', additional_kwargs={}, example=False),
  HumanMessage(content='print two random number', additional_kwargs={}, example=False),
  AIMessage(content='The two random numbers are 77 and 12.', additional_kwargs={}, example=False),
  HumanMessage(content='print two random number', additional_kwargs={}, example=False),
  AIMessage(content='The two random numbers are 31 and 17.', additional_kwargs={}, example=False),
  HumanMessage(content='print two random number', additional_kwargs={}, example=False),
  AIMessage(content='The two random n

In [10]:
agent_executor({"input": "can you request google.com and show me the logo in the website"})



> Entering new AgentExecutor chain...

Invoking: `python` with `import requests
from PIL import Image
from io import BytesIO

# Send a GET request to google.com
response = requests.get("https://www.google.com")

# Get the logo image URL from the response
logo_url = "https://www.google.com/images/branding/googlelogo/1x/googlelogo_color_272x92dp.png"

# Send a GET request to the logo image URL
logo_response = requests.get(logo_url)

# Create an Image object from the logo image data
logo_image = Image.open(BytesIO(logo_response.content))

# Display the logo image
logo_image.show()`


ModuleNotFoundError: No module named 'PIL'It seems that the `PIL` (Python Imaging Library) module is not installed in the current environment. This module is required to process and display the image.

To install the `PIL` module, you can use the following command:

```
!pip install pillow
```

Once the module is installed, you can run the code again to request google.com and display the logo image.

> Fini

{'input': 'can you request google.com and show me the logo in the website',
 'chat_history': [HumanMessage(content='create two random number, add them with 100', additional_kwargs={}, example=False),
  AIMessage(content='The sum of two random numbers, added with 100, is 230.', additional_kwargs={}, example=False),
  HumanMessage(content='create two random number, add them with 100', additional_kwargs={}, example=False),
  AIMessage(content='The sum of two random numbers, added with 100, is 160.', additional_kwargs={}, example=False),
  HumanMessage(content='print two random number', additional_kwargs={}, example=False),
  AIMessage(content='The two random numbers are 77 and 12.', additional_kwargs={}, example=False),
  HumanMessage(content='print two random number', additional_kwargs={}, example=False),
  AIMessage(content='The two random numbers are 31 and 17.', additional_kwargs={}, example=False),
  HumanMessage(content='print two random number', additional_kwargs={}, example=False)

In [ ]:
sandbox.run("!!pip install pillow")